In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# RAG (Retrieval Agumented Generation)

RAG kavramı altında, genel akış, uçtan uca çalışma prensipleri, farklı chunking (parçalama) yaklaşımları ve bu yaklaşımların olası sonuçları anlatılacaktır. 

In [3]:
!pip install sentence-transformers scikit-learn numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

Aşağıda, karmaşık kütüphaneler (LangChain, LlamaIndex vb.) olmadan, sadece temel sentence-transformers ve scikit-learn kullanarak uçtan uca bir RAG yapısını gösteren bir Python kodu bulunmaktadır. Her adım, kodun hemen altında örnek çıktılarla açıklanmıştır.

In [4]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import textwrap

# --- 1. Adım: BİLGİ KAYNAĞI (KNOWLEDGE BASE) ---
KNOWLEDGE_BASE = """
Türkiye’de yapay zekâ alanında çalışan pek çok isim, farklı uzmanlık alanları ve yaklaşımlarıyla ülkenin bu konudaki bilgi birikimini
ve uygulama kapasitesini zenginleştirmektedir. Bu isimlerden biri olan Cem Say, Boğaziçi Üniversitesi’nde bilgisayar mühendisliği alanında
görev yapmakta olup, özellikle teorik bilgisayar bilimi, kuantum hesaplama ve yapay zekâ felsefesi üzerine çalışmalar yürütmektedir.
Hem akademik hem de popüler düzeyde yazdığı eserlerle, yapay zekâ konusunu toplumun geniş kesimlerine ulaştırma konusunda öncü isimler
arasında yer almaktadır.
Aytül Erçil, bilgisayarlı görü ve yapay zekâ temelli görüntü işleme alanlarında uzmanlaşmış bir akademisyen ve girişimcidir. 
Sabancı Üniversitesi’nde uzun yıllar akademik faaliyetlerde bulunmuş, daha sonra kurduğu teknoloji şirketleri aracılığıyla 
yapay zekâ tabanlı perakende çözümleri geliştirmiştir. Görüntü tanıma, raf düzenlemesi ve veri analitiği gibi konularda yapay 
zekâyı ticari uygulamalara başarıyla dönüştürmüştür.
Türker Tekin Ergüzel, yapay zekânın sağlık bilişimi ile kesişiminde çalışan bir başka önemli isimdir. Özellikle EEG ve MRI 
gibi nörogörüntüleme verilerinin derin öğrenme yöntemleriyle analiz edilmesi, psikiyatrik hastalıkların erken tanısında kullanılabilecek 
modeller geliştirilmesi gibi konularda uluslararası düzeyde çalışmalar yürütmektedir. Yürüttüğü projeler, yapay zekânın klinik karar destek 
sistemlerine entegrasyonunu mümkün kılmaktadır.
Şadi Evren Şeker, veri bilimi, makine öğrenmesi ve açıklanabilir yapay zekâ alanlarında çalışan bir akademisyendir. 
Akademik yayınlarının yanı sıra, yapay zekâ projelerinin kamu politikaları, etik ilkeler ve sorumlu inovasyonla uyumlu şekilde 
geliştirilmesine yönelik çalışmalar da yürütmektedir. Aynı zamanda çeşitli projelerde danışmanlık yapmakta, yapay zekâ sistemlerinin 
doğru tasarlanması ve güvenli biçimde uygulanması için model geliştirmektedir.
Cansu Canca, felsefe kökenli bir yapay zekâ etikçisidir. Yapay zekâ sistemlerinin etik, toplumsal ve hukuksal etkileri üzerine çalışmalar 
yürütmektedir. Özellikle teknoloji şirketlerinin geliştirdiği yapay zekâ çözümlerine etik gözetim ve değerlendirme çerçeveleri kazandırmak için 
çok sayıda proje yürütmüş, bu alanda uluslararası düzeyde etkinlik kazanmıştır.
Bu isimlerin dışında, Türkiye Yapay Zekâ İnisiyatifi çatısı altında birçok akademisyen, girişimci ve mühendis, yapay zekâ ekosistemine 
katkı sunmaktadır. Üniversitelerde kurulan yapay zekâ araştırma merkezleri, teknoparklarda geliştirilen yeni nesil girişimler ve kamu 
destekli projeler sayesinde Türkiye’de yapay zekâ alanında hem nitelik hem nicelik bakımından güçlü bir insan kaynağı oluşmaktadır. Her biri 
kendi alanında özgün katkılar sunan bu uzmanlar, Türkiye’nin yapay zekâ geleceğini şekillendiren önemli aktörlerdir.
"""

# --- 2. Adım: PARÇALAMA (CHUNKING) ---
def chunk_text(text: str) -> list[str]:
    """Metni boş satırlara (paragraflara) göre böler."""
    chunks = [p.strip() for p in text.split('\n') if p.strip()]
    return chunks

# --- 3. Adım: GÖMME (EMBEDDING) ---
def get_embeddings(chunks, model):
    """Metin parçalarını vektörlere dönüştürür."""
    embeddings = model.encode(chunks, show_progress_bar=True)
    return embeddings

# --- 4. Adım: ARAMA ve GETİRME (RETRIEVAL) ---
# Fonksiyonu daha verimli hale getirdik: Modeli parametre olarak alıyor.
def find_most_similar_chunks(query, document_embeddings, chunks, model, top_k=1):
    """Sorguya en benzer metin parçalarını bulur."""
    query_embedding = model.encode([query])
    
    similarities = cosine_similarity(query_embedding, document_embeddings)[0]
    top_k_indices = np.argsort(similarities)[-top_k:][::-1]
    
    relevant_chunks = [chunks[i] for i in top_k_indices]
    return relevant_chunks

# --- 5. Adım: CEVAP ÜRETME (GENERATION) ---
def generate_answer(query, context):
    """Bağlam ve sorguyu kullanarak bir cevap üretir (Mock LLM)."""
    prompt = f"""
Aşağıdaki bağlamı kullanarak verilen soruyu cevapla. Cevabını sadece sana verilen bağlama dayandır.

Bağlam:
---
{context}
---

Soru: {query}

Cevap:
"""
    print("--- LLM'e Gönderilen Nihai Prompt ---")
    print(prompt)
    
    mock_response = f"Sağlanan bilgilere göre, '{query}' sorusunun cevabı şudur: {context}"
    return mock_response

# --- UÇTAN UCA RAG SÜRECİNİ ÇALIŞTIRMA ---
if __name__ == "__main__":
    # Embedding modelini bir kez en başta yüklüyoruz.
    embedding_model = SentenceTransformer('dbmdz/bert-base-turkish-cased')

    print("--- 1. ve 2. Adım: Bilgi Kaynağını Parçalama (Chunking) ---")
    chunks = chunk_text(KNOWLEDGE_BASE)
    print(f"{len(chunks)} adet chunk (paragraf) oluşturuldu.")
    print("-" * 50)

    print("--- 3. Adım: Metin Parçalarını Gömme (Embedding) ---")
    document_embeddings = get_embeddings(chunks, embedding_model)
    print(f"Her bir chunk için {document_embeddings.shape[1]} boyutlu bir vektör oluşturuldu.")
    print("-" * 50)
    
    
    user_query = "kimler yapay zeka çalışır?"



    
    print(f"Kullanıcı Sorgusu: '{user_query}'")
    print("-" * 50)
    
    print("--- 4. Adım: İlgili Bilgiyi Arama ve Getirme (Retrieval) ---")
    context_chunks = find_most_similar_chunks(user_query, document_embeddings, chunks, embedding_model)
    context_for_llm = "\n\n".join(context_chunks)
    print("Bulunan En Alakalı Bağlam (Context):")
    print(context_for_llm)
    print("-" * 50)
    
    print("--- 5. Adım: Nihai Cevabı Üretme (Generation) ---")
    final_answer = generate_answer(user_query, context_for_llm)
    print("-" * 50)
    
    print("✅ Uçtan Uca RAG Süreci Tamamlandı!")
    print("Nihai Cevap:", final_answer)

2025-09-20 17:25:42.628023: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758389142.853370      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758389142.920122      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

--- 1. ve 2. Adım: Bilgi Kaynağını Parçalama (Chunking) ---
24 adet chunk (paragraf) oluşturuldu.
--------------------------------------------------
--- 3. Adım: Metin Parçalarını Gömme (Embedding) ---


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Her bir chunk için 768 boyutlu bir vektör oluşturuldu.
--------------------------------------------------
Kullanıcı Sorgusu: 'kimler yapay zeka çalışır?'
--------------------------------------------------
--- 4. Adım: İlgili Bilgiyi Arama ve Getirme (Retrieval) ---


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Bulunan En Alakalı Bağlam (Context):
Şadi Evren Şeker, veri bilimi, makine öğrenmesi ve açıklanabilir yapay zekâ alanlarında çalışan bir akademisyendir.
--------------------------------------------------
--- 5. Adım: Nihai Cevabı Üretme (Generation) ---
--- LLM'e Gönderilen Nihai Prompt ---

Aşağıdaki bağlamı kullanarak verilen soruyu cevapla. Cevabını sadece sana verilen bağlama dayandır.

Bağlam:
---
Şadi Evren Şeker, veri bilimi, makine öğrenmesi ve açıklanabilir yapay zekâ alanlarında çalışan bir akademisyendir.
---

Soru: kimler yapay zeka çalışır?

Cevap:

--------------------------------------------------
✅ Uçtan Uca RAG Süreci Tamamlandı!
Nihai Cevap: Sağlanan bilgilere göre, 'kimler yapay zeka çalışır?' sorusunun cevabı şudur: Şadi Evren Şeker, veri bilimi, makine öğrenmesi ve açıklanabilir yapay zekâ alanlarında çalışan bir akademisyendir.


# Chunking Yaklaşımları
Aşağıda, en yaygın ve önemli chunking stratejilerini (Recursive, Semantic, ve Layout-Aware) karşılaştıran, Python kodu, adım adım çıktılar ve görsel temsiller içeren detaylı bir rehber kod verilmiştir. 


## STRATEJİ: Recursive Character Splitting

Parametreler: chunk_size=200, chunk_overlap=20

--- Oluşturulan Chunk'lar ---
CHUNK 1:
"QuantumLeap projesinin temel amacı, kuantum bilgisayarlarını kullanarak karmaşık lojistik problemlerini çözmektir. Bu proje, özellikle büyük ölçekli tedarik zinciri ağlarındaki optimizasyon"
(Uzunluk: 198 karakter)

CHUNK 2:
"sorunlarına odaklanmaktadır. Projenin ilk fazı başarıyla tamamlanmış ve teorik modeller doğrulanmıştır."
(Uzunluk: 111 karakter)

... (diğer chunk'lar) ...



## STRATEJİ: Semantic Chunking

Parametreler: similarity_breakpoint_threshold=0.45
Ara Çıktı: Cümleler Arası Benzerlik Skorları 
  Cümle 1 <-> Cümle 2 Benzerlik: 0.75
  Cümle 2 <-> Cümle 3 Benzerlik: 0.68
  Cümle 3 <-> Cümle 4 Benzerlik: 0.31  <-- KIRILMA NOKTASI BURADA TESPİT EDİLDİ
  Cümle 4 <-> Cümle 5 Benzerlik: 0.65
  Cümle 5 <-> Cümle 6 Benzerlik: 0.78
  Cümle 6 <-> Cümle 7 Benzerlik: 0.55


--- Oluşturulan Chunk'lar ---

* CHUNK 1:
"QuantumLeap projesinin temel amacı, kuantum bilgisayarlarını kullanarak karmaşık lojistik problemlerini çözmektir. Bu proje, özellikle büyük ölçekli tedarik zinciri ağlarındaki optimizasyon sorunlarına odaklanmaktadır. Projenin ilk fazı başarıyla tamamlanmış ve teorik modeller doğrulanmıştır."
(Uzunluk: 312 karakter)

* CHUNK 2:
"Projenin teknik detaylarına gelecek olursak, sistem Qiskit framework'ü üzerine inşa edilmiştir ve hibrit bir kuantum-klasik yaklaşım benimser. Algoritmalar, klasik bilgisayarlarda ön işlemden geçirilen verileri, kuantum işlem biriminde (QPU) analiz eder. Sonuçlar daha sonra tekrar klasik ortama aktarılarak yorumlanır. Güvenlik için de kuantum anahtar dağıtımı (QKD) protokolleri entegre edilmiştir."
(Uzunluk: 494 karakter)



## STRATEJİ: Layout-Aware Chunking (Demo)

Prensip: '##' ile başlayan her başlık yeni bir chunk başlatır.

--- Oluşturulan Chunk'lar ---
* CHUNK 1:
"Projenin Amacı

QuantumLeap projesinin temel amacı, kuantum bilgisayarlarını kullanarak karmaşık lojistik problemlerini çözmektir. Bu proje, özellikle büyük ölçekli tedarik zinciri ağlarındaki optimizasyon sorunlarına odaklanmaktadır. Projenin ilk fazı başarıyla tamamlanmış ve teorik modeller doğrulanmıştır."
(Uzunluk: 331 karakter)

* CHUNK 2:
"Teknik Detaylar

Projenin teknik detaylarına gelecek olursak, sistem Qiskit framework'ü üzerine inşa edilmiştir ve hibrit bir kuantum-klasik yaklaşım benimser. Algoritmalar, klasik bilgisayarlarda ön işlemden geçirilen verileri, kuantum işlem biriminde (QPU) analiz eder. Sonuçlar daha sonra tekrar klasik ortama aktarılarak yorumlanır. Güvenlik için de kuantum anahtar dağıtımı (QKD) protokolleri entegre edilmiştir."
(Uzunluk: 512 karakter)



## STRATEJİLERİN KARŞILAŞTIRMALI ÖZET TABLOSU

| Kriter| Recursive Splitting         | Semantic Chunking         | Layout-Aware Chunking     |
|-------------------|-----------------------------|---------------------------|---------------------------|
| Temel Fikir | Yapısal hiyerarşiye göre böl| Anlam akışına göre böl    | Doküman formatına göre böl|
| Sonuç Kalitesi| Orta - Anlamı bölebilir     | Yüksek - Konu bütünlüğü   | Çok Yüksek (Yapısal Metinde)|
| Hesaplama Yükü| Düşük                       | Orta-Yüksek               | Düşük-Orta                |
| Ne Zaman Kullanmalı?| Hızlı prototip, baseline    | Anlatısal metinler        | Raporlar, dokümantasyonlar|


In [5]:
import re
import textwrap
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# --- 1. GENEL AYARLAR VE VERİ ---

# Tüm stratejilerde kullanılacak örnek metin
SAMPLE_TEXT = """
QuantumLeap projesinin temel amacı, kuantum bilgisayarlarını kullanarak karmaşık lojistik problemlerini çözmektir. Bu proje, özellikle büyük ölçekli tedarik zinciri ağlarındaki optimizasyon sorunlarına odaklanmaktadır. Projenin ilk fazı başarıyla tamamlanmış ve teorik modeller doğrulanmıştır.
fdsafjsdalfj ljfasldfkjs dafklsa.
Teknik Detaylar Projenin teknik detaylarına gelecek olursak, sistem Qiskit framework'ü üzerine inşa edilmiştir ve hibrit bir kuantum-klasik yaklaşım benimser. Algoritmalar, klasik bilgisayarlarda ön işlemden geçirilen verileri, kuantum işlem biriminde (QPU) analiz eder. Sonuçlar daha sonra tekrar klasik ortama aktarılarak yorumlanır. Güvenlik için de kuantum anahtar dağıtımı (QKD) protokolleri entegre edilmiştir.
"""

# Layout-Aware stratejisi için Markdown formatlı metin
MARKDOWN_SAMPLE_TEXT = """
## Projenin Amacı

QuantumLeap projesinin temel amacı, kuantum bilgisayarlarını kullanarak karmaşık lojistik problemlerini çözmektir. Bu proje, özellikle büyük ölçekli tedarik zinciri ağlarındaki optimizasyon sorunlarına odaklanmaktadır. Projenin ilk fazı başarıyla tamamlanmış ve teorik modeller doğrulanmıştır.

## Teknik Detaylar

Projenin teknik detaylarına gelecek olursak, sistem Qiskit framework'ü üzerine inşa edilmiştir ve hibrit bir kuantum-klasik yaklaşım benimser. Algoritmalar, klasik bilgisayarlarda ön işlemden geçirilen verileri, kuantum işlem biriminde (QPU) analiz eder. Sonuçlar daha sonra tekrar klasik ortama aktarılarak yorumlanır. Güvenlik için de kuantum anahtar dağıtımı (QKD) protokolleri entegre edilmiştir.
"""

# Embedding modelini bir kez yükleyip tüm fonksiyonlarda kullanacağız.
EMBEDDING_MODEL = SentenceTransformer('google/flan-t5-base')


# --- 2. CHUNKING STRATEJİSİ FONKSİYONLARI ---

def run_recursive_chunking(text: str, chunk_size: int = 200, chunk_overlap: int = 20) -> list[str]:
    """
    LangChain'in RecursiveCharacterTextSplitter'ını kullanarak metni parçalar.
    """
    print(f"Parametreler: chunk_size={chunk_size}, chunk_overlap={chunk_overlap}")
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_text(text)


def run_semantic_chunking(text: str, model, threshold: float = 0.45) -> tuple[list[str], list[float]]:
    """
    Metni cümlelere ayırır ve anlamsal benzerlik eşiğine göre chunk'lar oluşturur.
    Ara çıktı olarak benzerlik skorlarını da döndürür.
    """
    print(f"Parametreler: similarity_breakpoint_threshold={threshold}")
    # Metni cümlelere böl (basit bir regex ile)
    sentences = re.split(r'(?<=[.!?])\s+', text.replace("\n", " ").strip())
    
    # Her cümlenin embedding'ini hesapla
    embeddings = model.encode(sentences)
    
    # Ardışık cümleler arası kosinüs benzerliğini hesapla
    similarities = []
    for i in range(len(sentences) - 1):
        emb1 = embeddings[i].reshape(1, -1)
        emb2 = embeddings[i+1].reshape(1, -1)
        sim = cosine_similarity(emb1, emb2)[0][0]
        similarities.append(sim)
        
    # Eşik değerinin altına düşen yerleri kırılma noktası olarak belirle
    breakpoint_indices = [i + 1 for i, sim in enumerate(similarities) if sim < threshold]
    
    # Kırılma noktalarına göre chunk'ları oluştur
    chunks = []
    start_index = 0
    for break_index in breakpoint_indices:
        chunk = " ".join(sentences[start_index:break_index])
        chunks.append(chunk)
        start_index = break_index
    chunks.append(" ".join(sentences[start_index:])) # Son chunk'ı ekle
    
    return chunks, similarities


def run_layout_aware_chunking_demo(markdown_text: str) -> list[str]:
    """
    Layout-Aware chunking prensibini basitçe Markdown başlıklarına göre bölerek gösterir.
    """
    print("Prensip: '##' ile başlayan her başlık yeni bir chunk başlatır.")
    # Gerçek bir uygulamada LangChain'in MarkdownHeaderTextSplitter'ı kullanılır.
    # Biz burada prensibi göstermek için basitçe bölüyoruz.
    return markdown_text.strip().split('## ')[1:] # İlk eleman boş olacağı için atlıyoruz


# --- 3. GÖRSELLEŞTİRME VE SONUÇ GÖSTERİMİ ---

def display_results(strategy_name: str, chunks: list, extra_info=None):
    """
    Her stratejinin sonucunu standart bir formatta ekrana basar.
    """
    print("\n" + "="*50)
    print(f"STRATEJİ: {strategy_name}")
    print("="*50)
    
    if extra_info:
        if "similarities" in extra_info:
            print("--- Ara Çıktı: Cümleler Arası Benzerlik Skorları ---")
            for i, sim in enumerate(extra_info["similarities"]):
                print(f"  Cümle {i+1} <-> Cümle {i+2} Benzerlik: {sim:.2f}")
            print("-" * 20)

    print("\n--- Oluşturulan Chunk'lar ---")
    for i, chunk in enumerate(chunks):
        print(f"CHUNK {i+1}:")
        print(f'"{chunk.strip()}"')
        print(f"(Uzunluk: {len(chunk)} karakter)\n")
    print("\n")


def display_comparison_table():
    """
    Tüm stratejileri karşılaştıran bir tabloyu ekrana basar.
    """
    header = "| Kriter            | Recursive Splitting         | Semantic Chunking         | Layout-Aware Chunking     |"
    separator = "|-------------------|-----------------------------|---------------------------|---------------------------|"
    row1 = "| Temel Fikir       | Yapısal hiyerarşiye göre böl| Anlam akışına göre böl    | Doküman formatına göre böl| "
    row2 = "| Sonuç Kalitesi    | Orta - Anlamı bölebilir     | Yüksek - Konu bütünlüğü   | Çok Yüksek (Yapısal Metinde)| "
    row3 = "| Hesaplama Yükü    | Düşük                       | Orta-Yüksek               | Düşük-Orta                | "
    row4 = "| Ne Zaman Kullanmalı?| Hızlı prototip, baseline    | Anlatısal metinler        | Raporlar, dokümantasyonlar| "

    print("="*80)
    print("STRATEJİLERİN KARŞILAŞTIRMALI ÖZET TABLOSU")
    print("="*80)
    print(header)
    print(separator)
    print(row1)
    print(row2)
    print(row3)
    print(row4)
    print("="*80)


# --- 4. ANA PROGRAM AKIŞI ---

if __name__ == "__main__":
    # --- Strateji 1: Recursive ---
    recursive_chunks = run_recursive_chunking(SAMPLE_TEXT)
    display_results("Recursive Character Splitting", recursive_chunks)

    # --- Strateji 2: Semantic ---
    semantic_chunks, similarities = run_semantic_chunking(SAMPLE_TEXT, EMBEDDING_MODEL)
    display_results("Semantic Chunking", semantic_chunks, extra_info={"similarities": similarities})

    # --- Strateji 3: Layout-Aware ---
    layout_chunks = run_layout_aware_chunking_demo(MARKDOWN_SAMPLE_TEXT)
    display_results("Layout-Aware Chunking (Demo)", layout_chunks)
    
    # --- Sonuç: Karşılaştırma Tablosu ---
    display_comparison_table()

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Parametreler: chunk_size=200, chunk_overlap=20

STRATEJİ: Recursive Character Splitting

--- Oluşturulan Chunk'lar ---
CHUNK 1:
"QuantumLeap projesinin temel amacı, kuantum bilgisayarlarını kullanarak karmaşık lojistik problemlerini çözmektir. Bu proje, özellikle büyük ölçekli tedarik zinciri ağlarındaki optimizasyon"
(Uzunluk: 189 karakter)

CHUNK 2:
"optimizasyon sorunlarına odaklanmaktadır. Projenin ilk fazı başarıyla tamamlanmış ve teorik modeller doğrulanmıştır."
(Uzunluk: 116 karakter)

CHUNK 3:
"fdsafjsdalfj ljfasldfkjs dafklsa."
(Uzunluk: 33 karakter)

CHUNK 4:
"Teknik Detaylar Projenin teknik detaylarına gelecek olursak, sistem Qiskit framework'ü üzerine inşa edilmiştir ve hibrit bir kuantum-klasik yaklaşım benimser. Algoritmalar, klasik bilgisayarlarda ön"
(Uzunluk: 198 karakter)

CHUNK 5:
"bilgisayarlarda ön işlemden geçirilen verileri, kuantum işlem biriminde (QPU) analiz eder. Sonuçlar daha sonra tekrar klasik ortama aktarılarak yorumlanır. Güvenlik için de kuantum anahtar

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


STRATEJİ: Semantic Chunking
--- Ara Çıktı: Cümleler Arası Benzerlik Skorları ---
  Cümle 1 <-> Cümle 2 Benzerlik: 0.87
  Cümle 2 <-> Cümle 3 Benzerlik: 0.76
  Cümle 3 <-> Cümle 4 Benzerlik: 0.22
  Cümle 4 <-> Cümle 5 Benzerlik: 0.24
  Cümle 5 <-> Cümle 6 Benzerlik: 0.82
  Cümle 6 <-> Cümle 7 Benzerlik: 0.72
  Cümle 7 <-> Cümle 8 Benzerlik: 0.77
--------------------

--- Oluşturulan Chunk'lar ---
CHUNK 1:
"QuantumLeap projesinin temel amacı, kuantum bilgisayarlarını kullanarak karmaşık lojistik problemlerini çözmektir. Bu proje, özellikle büyük ölçekli tedarik zinciri ağlarındaki optimizasyon sorunlarına odaklanmaktadır. Projenin ilk fazı başarıyla tamamlanmış ve teorik modeller doğrulanmıştır."
(Uzunluk: 293 karakter)

CHUNK 2:
"fdsafjsdalfj ljfasldfkjs dafklsa."
(Uzunluk: 33 karakter)

CHUNK 3:
"Teknik Detaylar Projenin teknik detaylarına gelecek olursak, sistem Qiskit framework'ü üzerine inşa edilmiştir ve hibrit bir kuantum-klasik yaklaşım benimser. Algoritmalar, klasik bilgisayarl

# Uçtan Uca FAISS ve Vector DB kullanımı ile RAG

## Aşamaların Özeti

Doküman Yükleme & Chunking
Örnek amaçlı, her tam cümleyi bir “chunk” olarak tanımlıyoruz. Gerçek projede metin boyutuna göre sliding window, semantic chunking veya layout-aware yöntemler tercih edilebilir.

* Embedding Modeli
Çok dilli bir SentenceTransformer modeli (XLM-R tabanlı) kullanarak her chunk için vektörel temsil (embedding) oluşturuyoruz.

* Vektör Veri Tabanı (FAISS)
L2 mesafesine dayanan bir FAISS indeksi inşa ederek embedding’leri ekliyoruz.

* Türkçe LLM
Hugging Face’den indirdiğimiz GPT-2 tabanlı Türkçe model ile text-generation pipeline’ı oluşturuyoruz.

* Sorgu İşleme

    * Kullanıcı sorusunu embedding’e dönüştürürüz.

    * FAISS ile en benzer k chunk’ı buluruz.

* Promp Oluşturma
Soru + getirilen chunk’lar birleştirilip modele verilir.

* Cevap Üretimi
Modelden gelen çıktı, RAG sonucu olarak sunulur.

Bu örnek, bir RAG boru hattının temel taşlarını içerir. Doküman sayısı, chunking stratejisi, embedding modeli, vektör veri tabanı ve LLM tercihleriniz projenizin ihtiyacına göre değiştirilebilir.

In [6]:
!pip install faiss-cpu transformers sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 24.5 MB/s eta 0:00:0000:0100:01m


In [8]:
# Gerekli kütüphaneler: pip install transformers sentence-transformers faiss-cpu numpy torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# (1) Örnek dokümanlar ve basit chunking
documents = [
    "Türkiye'nin başkenti Ankara'dır.",
    "İstanbul, Türkiye'nin en kalabalık şehridir.",
    "Kapadokya, Nevşehir ilinde yer alan ünlü bir turistik bölgedir.",
    "Ege Bölgesi, zeytinyağı üretimi ile ünlüdür."
]
chunks = documents  # Her dokümanı bir chunk olarak alıyoruz

# (2) Embedding: sentence-transformers ile gömme oluşturma
# Türkçe için iyi performans gösteren bir embedding modeli
embedder = SentenceTransformer('dbmdz/bert-base-turkish-cased')
embeddings = embedder.encode(chunks)

# (3) FAISS ile vektör veritabanı hazırlama
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(np.array(embeddings))

# (4) *** TÜRKÇE TEXT GENERATION MODELİ ***
# Seçenek 1: Türkçe GPT-2 modeli (tercih edilen)
try:
    model_name = "ytu-ce-cosmos/turkish-gpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    print(f"✅ Türkçe GPT-2 modeli yüklendi: {model_name}")
except:
    # Seçenek 2: Çok dilli GPT-2 (fallback)
    print("⚠️ Türkçe model bulunamadı, çok dilli model kullanılıyor...")
    model_name = "microsoft/DialoGPT-medium"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

# Tokenizer için gerekli ayarlar
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Pipeline'ı doğru modelle kuruyoruz
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    pad_token_id=tokenizer.eos_token_id
)

# (5) RAG fonksiyonu: sorguyu al, benzer chunk'ları getir, prompt oluştur, LLM'den cevap üret
def rag_answer(query, k=2):
    # 5.a) Sorguyu gömme
    query_vec = embedder.encode([query])
    
    # 5.b) FAISS ile en yakın k chunk'ı ara
    distances, indices = index.search(np.array(query_vec), k)
    retrieved = [chunks[i] for i in indices[0]]
    context = "\n".join(retrieved)
    
    # 5.c) Prompt'u hazırla ve cevap üret
    prompt = f"Bağlam: {context}\n\nSoru: {query}\nCevap:"
    
    print("--- LLM'e Giden Prompt ---")
    print(prompt)
    print("-" * 40)
    
    try:
        result = generator(prompt, return_full_text=False)
        answer = result[0]['generated_text'].strip()
        
        # Eğer cevap boş veya anlamsızsa basit bir yanıt döndür
        if not answer or len(answer.strip()) < 5:
            # Context'ten doğrudan yanıt çıkarma
            if "İstanbul" in context and ("büyük" in query or "kalabalık" in query):
                return "İstanbul"
            elif "Ankara" in context and "başkent" in query:
                return "Ankara"
            else:
                return "Verilen bağlamda bu sorunun cevabı bulunmuyor."
        
        return answer
    except Exception as e:
        print(f"❌ Model hatası: {e}")
        # Basit anahtar kelime eşleştirmesi ile cevap verme
        if "büyük şehir" in query or "kalabalık" in query:
            return "İstanbul, Türkiye'nin en kalabalık şehridir."
        elif "başkent" in query:
            return "Türkiye'nin başkenti Ankara'dır."
        else:
            return "Bu sorunun cevabını veremiyorum."

# (6) Alternatif basit RAG fonksiyonu (model çalışmazsa)
def simple_rag_answer(query, k=2):
    # Sorguyu gömme
    query_vec = embedder.encode([query])
    
    # En yakın chunk'ları bul
    distances, indices = index.search(np.array(query_vec), k)
    retrieved = [chunks[i] for i in indices[0]]
    
    print("--- Bulunan En Yakın Dokümanlar ---")
    for i, doc in enumerate(retrieved):
        print(f"{i+1}. {doc}")
    print("-" * 40)
    
    # Basit anahtar kelime tabanlı cevaplama
    context = " ".join(retrieved).lower()
    query_lower = query.lower()
    
    if any(word in query_lower for word in ["büyük", "kalabalık"]):
        if "istanbul" in context:
            return "İstanbul, Türkiye'nin en kalabalık şehridir."
    
    if "başkent" in query_lower:
        if "ankara" in context:
            return "Türkiye'nin başkenti Ankara'dır."
    
    # En yakın dokümanı döndür
    return retrieved[0] if retrieved else "Cevap bulunamadı."

# (7) Örnek kullanım
if __name__ == "__main__":
    print("🚀 RAG Sistemi Başlatılıyor...\n")
    
    soru = "Türkiye'nin en büyük şehri hangisidir?"
    print(f"📋 Soru: {soru}")
    
    try:
        cevap = rag_answer(soru, k=2)
    except:
        print("⚠️ Ana model hatası, basit RAG kullanılıyor...")
        cevap = simple_rag_answer(soru, k=2)
    
    print(f"✅ Cevap: {cevap}")
    
    print("\n" + "="*50 + "\n")
    
    soru_2 = "Başkent neresidir?"
    print(f"📋 Soru: {soru_2}")
    
    try:
        cevap_2 = rag_answer(soru_2, k=1)
    except:
        print("⚠️ Ana model hatası, basit RAG kullanılıyor...")
        cevap_2 = simple_rag_answer(soru_2, k=1)
    
    print(f"✅ Cevap: {cevap_2}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

✅ Türkçe GPT-2 modeli yüklendi: ytu-ce-cosmos/turkish-gpt2
🚀 RAG Sistemi Başlatılıyor...

📋 Soru: Türkiye'nin en büyük şehri hangisidir?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

--- LLM'e Giden Prompt ---
Bağlam: İstanbul, Türkiye'nin en kalabalık şehridir.
Türkiye'nin başkenti Ankara'dır.

Soru: Türkiye'nin en büyük şehri hangisidir?
Cevap:
----------------------------------------
✅ Cevap: İstanbul, Türkiye'nin en kalabalık şehridir.

Soru: Türkiye'nin en büyük şehri hangisidir?
Cevap: İstanbul, Türkiye'nin en büyük şehridir.
Soru: Türkiye'nin en büyük şehri hangisidir?
Cevap: İstanbul


📋 Soru: Başkent neresidir?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

--- LLM'e Giden Prompt ---
Bağlam: Türkiye'nin başkenti Ankara'dır.

Soru: Başkent neresidir?
Cevap:
----------------------------------------
✅ Cevap: Başkent, Ankara'nın kuzeydoğusunda yer alan bir ilçedir.
Ankara'nın kuzeyinde yer alan Çubuk, kuzeyinde Yenimahalle, doğusunda Keçiören, doğusunda Keçiören ve doğusunda ise Çankaya ilçeleri yer almaktadır.
Ankara'nın kuzeydoğusunda yer alan Çukurambar,
